In [1]:
#all imports
import tensorflow as tf
import torch

In [2]:

gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  raise(SystemError('GPU device not found'))

Found GPU at :/device:GPU:0


In [3]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print("There are %d GPU DEVICES available " %torch.cuda.device_count())
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")



There are 1 GPU DEVICES available 
The device name is Tesla P100-PCIE-16GB


In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Custom Data set and Data loader has been inspired from 
#https://github.com/sugi-chan/custom_bert_pipeline/blob/master/bert_pipeline.ipynb

In [6]:
!unzip -P ********** -qq '/content/drive/My Drive/DanishData/Danish.zip'

In [7]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a']
greekdata = pd.read_csv("Danish/offenseval-da-training-v1.tsv", delimiter='\t',names=headers)
data=greekdata[1:]


In [9]:
data=data[:2960]
print(data.head())
print(len(data))

     id                                              tweet subtask_a
1  3131  Jeg tror det vil være dejlig køligt, men jeg v...       NOT
2   711  Så kommer de nok til at investere i en ny cyke...       NOT
3  2500  Nu er det jo også de Ikea-aber der har lavet s...       OFF
4  2678  128 Varme emails, er vi enige om at det er sex...       NOT
5   784  Desværre tyder det på, at amerikanerne er helt...       NOT
2960


In [10]:
dfnumpy=data.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)

In [11]:
data.iloc[0:]

,id,tweet,subtask_a
1,3131,"Jeg tror det vil være dejlig køligt, men jeg v...",NOT
2,711,Så kommer de nok til at investere i en ny cyke...,NOT
3,2500,Nu er det jo også de Ikea-aber der har lavet s...,OFF
4,2678,"128 Varme emails, er vi enige om at det er sex...",NOT
5,784,"Desværre tyder det på, at amerikanerne er helt...",NOT
...,...,...,...
2956,170,Har sgu lidt en anelse om... det her kunne mås...,NOT
2957,1226,"Ind og ruske tremmer med hende,Den syge kælling!!",OFF
2958,2596,fedtmule,NOT
2959,1802,##HAR I HØRT DET?,NOT


In [12]:
print(len(dfnumpy))

2960


In [13]:
arrt=x[:,0]

In [14]:
arrt[0:3]

array(['Jeg tror det vil være dejlig køligt, men jeg vil have det meget meget svært, såfremt personen i billedet skulle lægge softicen der. ',
       'Så kommer de nok til at investere i en ny cykelpumpe, så landbetjenten kan rykke ud på sin cykel',
       'Nu er det jo også de Ikea-aber der har lavet spillet.. Det i sig selv gør det uspilleligt.'],
      dtype=object)

In [15]:
import re
def ft1(l):
    a = re.split('|\]|\[|\)|\(|; |,|\*|\n',l)
    li = []
    for t in a:
        li.append(t.strip(' ,')) 
    return li

In [16]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import string
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
def preprocessinglib(arrt):
    ans =[]
    for txt in arrt:
        ans.append(ft1(txt))
               
    return ans


In [17]:
#len(allTokens)
preprocessedTweets=arrt

In [18]:

from sklearn.utils import shuffle
preprocessedTweets, y = shuffle(preprocessedTweets, y)

In [19]:
print(preprocessedTweets[0:3])

['Amen'
 'Hendes majestæt dronning Margrethe D. 2 af Danmark længe leve...!!! Ændret fra hendes kongelige højhed til hendes majestæt, fordi jeg har bragt skam over vore kære Danmark, ved ikke at titulere majestæten korrekt.'
 'Folk siger det er en spøg/sarkastisk. Jeg tvivler stærkt.']


In [20]:
from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

Using TensorFlow backend.


In [21]:
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)


In [22]:
sentence_train, sentence_test, y_train, y_test = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42)
#x_train_mask,x_test_mask,_,_=train_test_split(attention_masks,y, test_size=0.2, random_state=42)



In [23]:
##do this at the time of training only 
#sentence_train, sentence_test1, y_train, y_test1 = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42) only this for code submission
#sentence_test,sentence_predict, y_test,y_predict = train_test_split(sentence_test1,y_test1, test_size=0.5, random_state=42)
#x_predict_mask,_,_,_=train_test_split(x_test_mask,y_test, test_size=0.5, random_state=42)

In [24]:
sentence_train[0]

'Hvis vi skal se positivt på det her, og det skal vi jo, så er jeg lykkelig, da blæsten var så venlig at tage genboens møgirriterende vindklokke med sig.'

In [25]:

from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

print(y_train.shape)
print(y_test.shape)
yTrain=le.fit_transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)


(2368, 1)
(592, 1)
(2368,)
['NOT' 'OFF']


In [26]:
yTest=le.transform(y_test.flatten())


In [27]:
set(y_test.flatten())

{'NOT', 'OFF'}

In [28]:
set(yTest)

{0, 1}

In [29]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        
        
    def __len__(self):
        return len(self.xytrain[0])

In [30]:
#from torch.utils.data import Dataset
#tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength-2]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])

In [31]:
xytrain=[sentence_train,yTrain]
tdataset = GreekTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)

In [32]:
xytest=[sentence_test,yTest]
tedataset = GreekTestDataset(xytest)
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

In [ ]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

In [34]:
params=list(model.named_parameters())

In [35]:
optimizer=AdamW(model.parameters(),lr=2e-5,eps=1e-8)
#############CAN CHANGE LEARNING RATE 

In [36]:
from transformers import get_linear_schedule_with_warmup

epochs=4
total_steps=len(tdataloader)*epochs

sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [37]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("predictionArgmax",predictionArgmax)
  print("labelsFlattend",labelsFlattend)
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)

In [38]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

epochs=4
lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    
    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()        
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])

        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()

    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
   
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0

    for batch in tedataloader:       
        batch = tuple(t.to(device) for t in batch)        
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():        
            outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))

    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
    
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  74.    Elapsed time is  11.635800838470459
 The training loss incured is  0.394
  Training one epoch time taken 17.041118144989014
 Validation starts here 
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 0 1 0 0 0 0 1 0 0 

In [39]:
!unzip -P ****** -qq '/content/drive/My Drive/DanishData/DanishPredict.zip'
!unzip -P ****** -qq '/content/drive/My Drive/DanishData/Danish.zip'

replace Danish/offenseval-da-training-v1.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Danish/readme-trainingset-da.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [40]:
ygiven=[]
ypredicted=[]

def convertToInt(val):
    if not val:
        return 0    
    try:
        return np.int64(val)
    except:        
        return np.int64(0)



In [41]:
##Use blob
#GET THE DATA FROM THE PANDAS FRAME
def readData1():
  headers=['id','tweet']
  greekDataTest = pd.read_csv("offenseval-da-test-v1-nolabels.tsv", delimiter='\t',names=headers,quoting=csv.QUOTE_NONE,
                                converters={"id":convertToInt})
  greekDataTest=greekDataTest[1:]
  print(greekDataTest.head())
  print(greekDataTest.dtypes)
  print(greekDataTest.shape)
  
  dfnumpy=greekDataTest.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  tid=dfnumpy[:, 0].reshape(-1, 1)
  print(tid)
  arrt=X[:,0]
  #allTokens=preprocess1(arrt)
  preprocessedTweets=arrt
  return preprocessedTweets,tid

In [ ]:
preprocessedTweets,tid=readData1()


In [43]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekPredictDataset(Dataset):
    def __init__(self,xypredict):
        self.xypredict = xypredict
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xypredict[0][index]))
        if len(tokenized_review) > self.maxlength:
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_predict_pytorch = torch.tensor(ids_of_sentence_word)
        x_predict_mask_pytorch=torch.tensor(attention_mask)
        tid_predict_pytorch=torch.tensor(self.xypredict[1][index])
        
        return x_predict_pytorch,x_predict_mask_pytorch,tid_predict_pytorch
       
    def __len__(self):
        return len(self.xypredict[0])
 


In [44]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def calculateScore(predictions):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  yres.append(predictionArgmax.flatten())
  print("predictionArgmax",predictionArgmax)
 

In [45]:
yres=[]
finalTid=[]
def predictingData(pTweets,tid):
  #https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP#scrollTo=1M296yz577fV
  ids_of_sentence=[]
  predictedLabels,trueLabels=[],[]
  
  map_location=""
  xypredict=[pTweets,tid.flatten()]
  
  tdataset = GreekPredictDataset(xypredict)
  tsampler=RandomSampler(tdataset)
  predictdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
  print(device.type)
  model.eval()
  i=0
  for batch in predictdataloader:
      print(i)
      i=i+1
      batch = tuple(t.to(device) for t in batch)        
      inputs = {"input_ids": batch[0], "attention_mask": batch[1], "tids":batch[2]}
      
      with torch.no_grad():       
          outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictedLabels.append(logits)
      tidl=(inputs["tids"]).to('cpu').numpy()
      finalTid.append(tidl)
      calculateScore(logits)
      
  return predictedLabels,finalTid

In [ ]:
preprocessedTweets,tid=readData1()

In [ ]:
predictedLabels,finalTid=predictingData(preprocessedTweets,tid)

In [48]:
yans=[yres[i].flatten().tolist() for i in range(len(yres))]
ytid=[finalTid[i].flatten().tolist() for i in range(len(finalTid))]

In [49]:
from itertools import chain

yans=list(chain.from_iterable(yans))
ytid=list(chain.from_iterable(ytid))

In [50]:
len(ytid)

329

In [51]:
yans1=["NOT" if yans[i]==0 else "OFF" for i in range(len(yans))]

In [52]:
print(yans1.count("OFF"))
print(yans1.count("NOT"))

18
311


In [53]:
#z={'tweet':sentence_predict[],'subtask_a':y_predict}
#print(len(z))
C = {'id': ytid,
        'predicted': yans1,
    }
df = pd.DataFrame(C)
print (df[df['id'] == 400] )

export_csv = df.to_csv ('/content/drive/My Drive/DanishData/AfterDanishAnswertest.csv', index = None, header=False)

     id predicted
63  400       NOT


In [54]:
headers=['id','ytrue']
ytrue = pd.read_csv("/content/drive/My Drive/SemevalAnalysis/DanishAnswer/danish-goldlabels.csv", delimiter=',',names=headers)

In [55]:
len(ytrue)

329

In [56]:
headers=['id','ypredicted']
ypredicted = pd.read_csv("/content/drive/My Drive/DanishData/AfterDanishAnswer.csv", delimiter=',',names=headers)

In [57]:
print(len(ypredicted))
print(ypredicted.head)

329
<bound method NDFrame.head of        id ypredicted
0     757        NOT
1     651        NOT
2    1189        NOT
3    1996        NOT
4    1375        NOT
..    ...        ...
324  1886        NOT
325  2449        NOT
326  3494        NOT
327  1558        NOT
328   386        NOT

[329 rows x 2 columns]>


In [58]:
result = pd.merge(ytrue, ypredicted, on='id', how='inner')
print(result.head())
print(result.dtypes)
print(result.shape)


     id ytrue ypredicted
0  1382   NOT        NOT
1  1384   NOT        NOT
2   547   NOT        NOT
3  1269   NOT        NOT
4  1695   OFF        NOT
id             int64
ytrue         object
ypredicted    object
dtype: object
(329, 3)


In [59]:

dfnumpy=result.to_numpy();
yidnumpy=dfnumpy[:, 0].reshape(-1, 1)
ytruenumpy=dfnumpy[:, 1].reshape(-1, 1)
ypredictednumpy=dfnumpy[:, 2].reshape(-1, 1)
#ytweetnumpy=dfnumpy[:, 3].reshape(-1, 1)


In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


cnf=confusion_matrix(ytruenumpy, ypredictednumpy, labels=['NOT', 'OFF'])
print(cnf)

df_cm = pd.DataFrame(cnf,['NOT', 'OFF'],['NOT', 'OFF'])
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True,fmt='.2f') # font size
plt.xlabel("predicted Labels")
plt.ylabel("true Labels")
plt.show()

In [61]:
from sklearn.metrics import f1_score

In [62]:
f1_score(ytruenumpy, ypredictednumpy, average='macro')

0.7103641456582632

In [63]:
torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/DanishData/submitdanishwopre64test.pth.tar')

In [64]:
headers=['id','ytrue']
ytrue = pd.read_csv("/content/drive/My Drive/SemevalAnalysis/DanishAnswer/danish-goldlabels.csv", delimiter=',',names=headers)

In [65]:
len(ytrue)

329

In [66]:
headers=['id','ypredicted']
ypredicted = pd.read_csv("/content/drive/My Drive/SemevalAnalysis/DanishAnswer/DanishAnswer.csv", delimiter=',',names=headers)

In [67]:
print(len(ypredicted))
print(ypredicted.head)

329
<bound method NDFrame.head of        id ypredicted
0     757        NOT
1     651        NOT
2    1189        NOT
3    1996        NOT
4    1375        NOT
..    ...        ...
324  1886        NOT
325  2449        NOT
326  3494        NOT
327  1558        NOT
328   386        NOT

[329 rows x 2 columns]>


In [68]:
result = pd.merge(ytrue, ypredicted, on='id', how='inner')
print(result.head())
print(result.dtypes)
print(result.shape)


     id ytrue ypredicted
0  1382   NOT        NOT
1  1384   NOT        NOT
2   547   NOT        NOT
3  1269   NOT        NOT
4  1695   OFF        NOT
id             int64
ytrue         object
ypredicted    object
dtype: object
(329, 3)


In [69]:

dfnumpy=result.to_numpy();
yidnumpy=dfnumpy[:, 0].reshape(-1, 1)
ytruenumpy=dfnumpy[:, 1].reshape(-1, 1)
ypredictednumpy=dfnumpy[:, 2].reshape(-1, 1)
#ytweetnumpy=dfnumpy[:, 3].reshape(-1, 1)


In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


cnf=confusion_matrix(ytruenumpy, ypredictednumpy, labels=['NOT', 'OFF'])
print(cnf)

df_cm = pd.DataFrame(cnf,['NOT', 'OFF'],['NOT', 'OFF'])
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True,fmt='.2f') # font size
plt.xlabel("predicted Labels")
plt.ylabel("true Labels")
plt.show()

In [71]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(ytruenumpy, ypredictednumpy, average='macro')